<a href="https://colab.research.google.com/github/victormmoraes/imersao-alura-ia/blob/main/Metr%C3%B4_inteligente_Imers%C3%A3o_IA_Alura_e_Google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# Instalando as dependências
!pip install googlemaps
!pip install unidecode
!pip install google-search-results
!pip install -q -U google-generativeai
!pip install googletrans

In [2]:
# Importações
import pandas as pd
import googlemaps
import google.generativeai as genai
from unidecode import unidecode
from serpapi import GoogleSearch
from googletrans import Translator
from google.cloud import translate_v2 as translate

In [3]:
# Acessando chave secreta no Google Colab
from google.colab import userdata
API_KEY = userdata.get('API_KEY')

In [4]:
# Define a API Key para o genai
genai.configure(api_key=API_KEY)

In [31]:
generation_config = {
  "candidate_count": 1,
  "temperature": 0,
}

safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                              generation_config=generation_config,
                              safety_settings=safety_settings,)

In [5]:
# Inicializa o cliente da API do Google Maps
gmaps = googlemaps.Client(key=API_KEY)

In [6]:
# Caminho para o arquivo da planilha no Google Colab
spreadsheet_path = '/content/metro_sp.xlsx'  # Substitua pelo nome do seu arquivo

In [8]:
# Carrega a planilha em um DataFrame
df = pd.read_excel(spreadsheet_path)

In [9]:
# Imprime o DataFrame
df

,nome_da_estacao,linha
0,Luz,Linha 1 - Azul
1,São Bento,Linha 1 - Azul
2,Sé,"Linha 1 - Azul, Linha 3 - Vermelha"
3,Liberdade,Linha 1 - Azul
4,Paraíso,"Linha 1 - Azul, Linha 2 - Verde"
...,...,...
77,AACD-Servidor,Linha 5 - Lilás
78,Santa Cruz,Linha 5 - Lilás
79,Chácara Klabin,Linha 5 - Lilás
80,Campo Belo,Linha 5 - Lilás


In [10]:
# Cria as colunas de Longitude e Latitude
df['Longitude'] = ''
df['Latitude'] = ''

In [11]:
# Itera pelas linhas do DataFrame e preenche as coordenadas
for index, row in df.iterrows():
    station_name = row['nome_da_estacao']

    # Faz a geocodificação usando a API do Google Maps
    geocode_result = gmaps.geocode(station_name + ', São Paulo, Brasil')

    # Verifica se a geocodificação foi bem-sucedida
    if geocode_result:
        lat = geocode_result[0]['geometry']['location']['lat']
        lng = geocode_result[0]['geometry']['location']['lng']
        df.loc[index, ['Latitude', 'Longitude']] = [lat, lng]
    else:
        print(f'Erro ao geocodificar a estação: {station_name}')

In [12]:
# Imprime o DataFrame com as coordenadas
print(df)

        nome_da_estacao                               linha  Longitude  \
0                   Luz                      Linha 1 - Azul  -46.62915   
1             São Bento                      Linha 1 - Azul -46.650397   
2                    Sé  Linha 1 - Azul, Linha 3 - Vermelha -46.630715   
3             Liberdade                      Linha 1 - Azul -46.631259   
4               Paraíso     Linha 1 - Azul, Linha 2 - Verde -46.648786   
..                  ...                                 ...        ...   
77        AACD-Servidor                     Linha 5 - Lilás -46.652281   
78           Santa Cruz                     Linha 5 - Lilás -46.636833   
79       Chácara Klabin                     Linha 5 - Lilás -46.629509   
80           Campo Belo                     Linha 5 - Lilás  -46.67202   
81  Comendador Ermelino                    Linha 15 - Prata -46.482337   

     Latitude  
0  -23.529456  
1  -23.502731  
2  -23.552849  
3  -23.559917  
4   -23.57476  
..        ...  

In [13]:
# Salva o DataFrame atualizado em um novo arquivo Excel (opcional)
df.to_excel('/content/metro_sp_coordenadas.xlsx', index=False)

In [14]:
# Cria um dicionário para mapear as cores das linhas
cores_linhas = {
    'Linha 1 - Azul': 'blue',
    'Linha 2 - Verde': 'green',
    'Linha 3 - Vermelha': 'red',
    'Linha 4 - Amarela': 'yellow',
    'Linha 5 - Lilás': 'purple',
    'Linha 15 - Prata': 'gray'
}

In [15]:
# Cria uma cópia do DataFrame
df_linhas = df.copy()


In [16]:
# Encontra a estação no DataFrame com base na comparação direta de strings, ignorando maiúsculas e minúsculas e caracteres especiais..

def encontrar_estacao(prompt, df):
  """
  Args:
    prompt: O nome da estação digitado pelo usuário.
    df: O DataFrame contendo os dados das estações.

  Returns:
    A linha do DataFrame correspondente à estação encontrada, ou None se não encontrar.
  """
  prompt = unidecode(prompt).lower()
  for index, row in df.iterrows():
    nome_estacao = unidecode(row['nome_da_estacao']).lower()
    if prompt == nome_estacao:
      return row
  return None

In [17]:
# Função para obter a estação validada do usuário
def obter_estacao_validada(tipo_estacao, df):
  while True:
    nome_da_estacao = input(f'Qual a estação de {tipo_estacao}? ')
    estacao_validada = encontrar_estacao(nome_da_estacao, df)
    if estacao_validada is not None:
      return estacao_validada
    print(f"Estação '{nome_da_estacao}' não encontrada. Tente novamente.")


In [18]:
# Estima o tempo de trajeto entre duas coordenadas usando a API do Google Maps.
def estimar_tempo_trajeto(origem, destino):
  """
  Args:
    origem: Array (latitude, longitude) da estação de origem.
    destino: Array (latitude, longitude) da estação de destino.

  Returns:
    O tempo estimado de trajeto em minutos, ou None se houver erro na consulta à API.
  """
  try:
    directions_result = gmaps.directions(
        origin=origem,
        destination=destino,
        mode="transit",  # Define o modo de transporte como transporte público
        departure_time="now"  # Define a hora de partida como agora
    )
    # Extrai o tempo de trajeto da resposta da API
    tempo_em_segundos = directions_result[0]['legs'][0]['duration']['value']
    tempo_em_minutos = tempo_em_segundos // 60
    return tempo_em_minutos
  except Exception as e:
    print(f"Erro ao estimar o tempo de trajeto: {e}")
    return None

In [19]:
# Calcula a emissão de CO2 da viagem de metrô
def calcular_emissao_co2_metro(distancia_km):
  emissao_co2_por_km = 0.025  # kg CO2 por km por passageiro (estimativa)
  emissao_total = distancia_km * emissao_co2_por_km
  return emissao_total

In [41]:
# Gera um texto sobre o impacto ambiental usando o genai
def gerar_texto_impacto_ambiental(emissao_co2):
  prompt = f"""
  Você está usando o metrô de São Paulo e evitou a emissão de {emissao_co2} kg de CO2.
  Crie um texto curto que inspire sobre a importância de fazer escolhas sustentáveis para o meio ambiente.
  Mencione como pequenas ações podem fazer a diferença e incentive outras pessoas a optarem pelo transporte público.
  """

  response = model.generate_content(prompt)

  return response.text

In [44]:
# Obtém os principais pontos turísticos próximos à estação de destino usando o genai.
def obter_pontos_turisticos(estacao_destino):
  prompt = f"""
  Liste os principais pontos turísticos próximos à estação de metrô {estacao_destino} em São Paulo.
  """

  response = model.generate_content(prompt)


  return response.text

In [34]:
# Obtém a estação de embarque validada
embarqueValidado = obter_estacao_validada('embarque', df)

Qual a estação de embarque? luz


In [24]:
# Obtém a estação de desembarque validada
desembarqueValidado = obter_estacao_validada('desembarque', df)

Qual a estação de desembarque? jabaquara


In [25]:
# Extrai as coordenadas das estações
origem = (embarqueValidado['Latitude'], embarqueValidado['Longitude'])
destino = (desembarqueValidado['Latitude'], desembarqueValidado['Longitude'])

In [26]:
# Estima o tempo de trajeto
tempo_estimado = estimar_tempo_trajeto(origem, destino)

if tempo_estimado is not None:
  print(f"Tempo estimado de trajeto: {tempo_estimado} minutos")

Tempo estimado de trajeto: 52 minutos


In [27]:
# Calcula a distância entre as estações usando a API do Google Maps
distancia_metros = gmaps.distance_matrix(
    origins=origem,
    destinations=destino,
    mode="transit"
)['rows'][0]['elements'][0]['distance']['value']

distancia_km = distancia_metros / 1000

In [36]:
# Calcula a emissão de CO2
emissao_co2 = calcular_emissao_co2_metro(distancia_km)

In [42]:
# Gera o texto sobre o impacto ambiental
texto_impacto = gerar_texto_impacto_ambiental(emissao_co2)
print(texto_impacto)

Ao optar pelo metrô, você não apenas economizou tempo e dinheiro, mas também contribuiu para um futuro mais verde. Cada viagem de metrô evita a emissão de 0,347 kg de CO2, reduzindo nossa pegada de carbono.

Pequenas ações, como escolher o transporte público, podem ter um impacto significativo no meio ambiente. Ao fazer escolhas sustentáveis, estamos protegendo nosso planeta para as gerações futuras. Junte-se a nós na jornada para um futuro mais limpo e opte pelo transporte público sempre que possível. Juntos, podemos fazer a diferença!


In [45]:
# Gera o texto sobre o impacto ambiental
texto_turistico = obter_pontos_turisticos(desembarqueValidado)
print(texto_turistico)

* **Parque Ibirapuera:** Um dos maiores e mais importantes parques urbanos da América Latina, com amplas áreas verdes, lagos, museus e outras atrações.
* **Museu Afro Brasil:** Dedicado à cultura e história afro-brasileira, com uma vasta coleção de arte, documentos e objetos.
* **Museu de Arte Moderna de São Paulo (MAM):** Um dos museus de arte moderna mais importantes do Brasil, com uma coleção que abrange obras de artistas brasileiros e internacionais.
* **Estádio do Morumbi:** O estádio do São Paulo Futebol Clube, um dos maiores e mais tradicionais clubes de futebol do Brasil.
* **Shopping Morumbi:** Um dos maiores e mais completos shoppings centers de São Paulo, com uma ampla variedade de lojas, restaurantes e opções de entretenimento.
* **Parque da Independência:** Um parque histórico que abriga o Museu do Ipiranga, onde foi proclamada a independência do Brasil.
* **Zoológico de São Paulo:** Um dos maiores e mais completos zoológicos da América Latina, com uma grande variedade de 